Скрипт позволяет из каждого движения выбрать интересующий временной участок и записать на диск изменённые файлы для скоростей, активаций и сил

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import random

In [5]:
#Описание типов движения по индексу
motiontype = {}
motiontype[1] = 'вращение головы вправо'
motiontype[2] = 'наклон головы вперёд'
motiontype[3] = 'наклон головы вправо'
motiontype[4] = 'наклон головы вправо до упора с плечом'
motiontype[5] = 'вращение головой против часовой'
motiontype[7] = 'плечо: поднятие вперёд'
motiontype[8] = 'плечо: абдукция'
motiontype[9] = 'пожимание двумя плечами'
motiontype[10] = 'вращение плеча саггитально'
motiontype[11] = 'вращение вокруг плечевой кости'
motiontype[12] = 'наклон головы вперёд'
motiontype[14] = 'плечо: вращение траверсально'

#Группы мышц
muscle_groups = {}
muscle_groups['Trapezius_R'] = ['trap_cl', 'trap_acr_scap', 'trap_acr_T1', 'trap_acr_T2', 'trap_acr_T3',
                                'trap_inf_T4', 'trap_inf_T5', 'trap_inf_T6', 'trap_inf_T7', 'trap_inf_T8', 'trap_inf_T9',
                                'trap_inf_T10', 'trap_inf_T11', 'trap_inf_T12']
muscle_groups['Deltoid_R'] = ['DELT1', 'DELT2', 'DELT3']
muscle_groups['Supraspinatus_R'] = ['SUPSP']
muscle_groups['Infraspinatus_R'] = ['INFSP']
muscle_groups['Subscapularis_R'] = ['SUBSC']
muscle_groups['Teres_Minor_R'] = ['TMIN']
muscle_groups['Teres_Major_R'] = ['TMAJ']
muscle_groups['Sternocleidomastoideus_R'] = ['stern_mast', 'cleid_mast', 'cleid_occ']
muscle_groups['Scalenus_R'] = ['scalenus_ant', 'scalenus_med', 'scalenus_post']
muscle_groups['Levator_Spapulae_R'] = ['levator_scap1', 'levator_scap2', 'levator_scap3', 'levator_scap4']
muscle_groups['Rhomboid_R'] = ['rhomb_C7', 'rhomb_T1', 'rhomb_T2', 'rhomb_T3', 'rhomb_T4', 'rhomb_T5']
muscle_groups['Obliqus Capitis_R'] = ['obl_cap_sup','obl_cap_inf']
muscle_groups['Rectus Capitis_R'] = ['rectcap_post_maj', 'rectcap_post_min', 'rect_cap_lat']
muscle_groups['Longissimus_R'] = ['longissi_cerv_c4thx', 'longissi_cap_c6']
muscle_groups['Sternohyoid_R'] = ['sternohyoid']
muscle_groups['Omohyoid_R'] = ['omohyoid_inf', 'omohyoid_sup']
muscle_groups['Longus colli_R'] = ['long_col_c1thx']
#Названия колонок для файлов активации и скорости
muscles_listav = ['time', 'trap_cl', 'trap_acr_scap', 'trap_acr_T1', 'trap_acr_T2', 'trap_acr_T3', 'trap_inf_T4', 'trap_inf_T5', 'trap_inf_T6', 'trap_inf_T7', 'trap_inf_T8', 'trap_inf_T9', 'trap_inf_T10', 'trap_inf_T11', 'trap_inf_T12', 'trap_cl_L', 'trap_acr_scap_L', 'trap_acr_T1_L', 'trap_acr_T2_L', 'trap_acr_T3_L', 'trap_inf_T4_L', 'trap_inf_T5_L', 'trap_inf_T6_L', 'trap_inf_T7_L', 'trap_inf_T8_L', 'trap_inf_T9_L', 'trap_inf_T10_L', 'trap_inf_T11_L', 'trap_inf_T12_L', 'SerrAnt2_1_R', 'SerrAnt2_2_R', 'SerrAnt3_1_R', 'SerrAnt4_1_R', 'SerrAnt5_1_R', 'SerrAnt6_1_R', 'SerrAnt7_1_R', 'SerrAnt8_1_R', 'SerrAnt9_1_R', 'SerrAnt2_1_L', 'SerrAnt2_2_L', 'SerrAnt3_1_L', 'SerrAnt4_1_L', 'SerrAnt5_1_L', 'SerrAnt6_1_L', 'SerrAnt7_1_L', 'SerrAnt8_1_L', 'SerrAnt9_1_L', 'DELT1', 'DELT2', 'DELT3', 'DELT1_l', 'DELT2_l', 'DELT3_l', 'SUPSP', 'SUPSP_l', 'INFSP', 'INFSP_l', 'SUBSC', 'SUBSC_l', 'TMIN', 'TMIN_l', 'TMAJ', 'TMAJ_l', 'stern_mast', 'cleid_mast', 'stern_mast_L', 'cleid_mast_L', 'cleid_occ', 'cleid_occ_L', 'scalenus_ant', 'scalenus_med', 'scalenus_ant_L', 'scalenus_med_L', 'scalenus_post', 'scalenus_post_L', 'levator_scap1', 'levator_scap2', 'levator_scap3', 'levator_scap4', 'levator_scap1_L', 'levator_scap2_L', 'levator_scap3_L', 'levator_scap4_L', 'rhomb_C7', 'rhomb_T1', 'rhomb_T2', 'rhomb_T3', 'rhomb_T4', 'rhomb_T5', 'rhomb_C7_L', 'rhomb_T1_L', 'rhomb_T2_L', 'rhomb_T3_L', 'rhomb_T4_L', 'rhomb_T5_L', 'sternohyoid', 'sternohyoid_L', 'omohyoid_inf', 'omohyoid_inf_L', 'omohyoid_sup', 'omohyoid_sup_L', 'rectcap_post_maj', 'rectcap_post_maj_L', 'rectcap_post_min', 'rectcap_post_min_L', 'obl_cap_sup', 'obl_cap_sup_L', 'obl_cap_inf', 'obl_cap_inf_L', 'long_col_c1thx', 'long_col_c1thx_L', 'longissi_cerv_c4thx', 'longissi_cerv_c4thx_L', 'longissi_cap_c6', 'longissi_cap_c6_L', 'long_cap_c4', 'long_cap_c4_L', 'rect_cap_lat', 'rect_cap_lat_L']
#Названия колонок для файлов сил
muscles_listf = ['time', 'trap_cl', 'trap_acr_scap', 'trap_acr_T1', 'trap_acr_T2', 'trap_acr_T3', 'trap_inf_T4', 'trap_inf_T5', 'trap_inf_T6', 'trap_inf_T7', 'trap_inf_T8', 'trap_inf_T9', 'trap_inf_T10', 'trap_inf_T11', 'trap_inf_T12', 'trap_cl_L', 'trap_acr_scap_L', 'trap_acr_T1_L', 'trap_acr_T2_L', 'trap_acr_T3_L', 'trap_inf_T4_L', 'trap_inf_T5_L', 'trap_inf_T6_L', 'trap_inf_T7_L', 'trap_inf_T8_L', 'trap_inf_T9_L', 'trap_inf_T10_L', 'trap_inf_T11_L', 'trap_inf_T12_L', 'SerrAnt2_1_R', 'SerrAnt2_2_R', 'SerrAnt3_1_R', 'SerrAnt4_1_R', 'SerrAnt5_1_R', 'SerrAnt6_1_R', 'SerrAnt7_1_R', 'SerrAnt8_1_R', 'SerrAnt9_1_R', 'SerrAnt2_1_L', 'SerrAnt2_2_L', 'SerrAnt3_1_L', 'SerrAnt4_1_L', 'SerrAnt5_1_L', 'SerrAnt6_1_L', 'SerrAnt7_1_L', 'SerrAnt8_1_L', 'SerrAnt9_1_L', 'DELT1', 'DELT2', 'DELT3', 'DELT1_l', 'DELT2_l', 'DELT3_l', 'SUPSP', 'SUPSP_l', 'INFSP', 'INFSP_l', 'SUBSC', 'SUBSC_l', 'TMIN', 'TMIN_l', 'TMAJ', 'TMAJ_l', 'stern_mast', 'cleid_mast', 'stern_mast_L', 'cleid_mast_L', 'cleid_occ', 'cleid_occ_L', 'scalenus_ant', 'scalenus_med', 'scalenus_ant_L', 'scalenus_med_L', 'scalenus_post', 'scalenus_post_L', 'levator_scap1', 'levator_scap2', 'levator_scap3', 'levator_scap4', 'levator_scap1_L', 'levator_scap2_L', 'levator_scap3_L', 'levator_scap4_L', 'rhomb_C7', 'rhomb_T1', 'rhomb_T2', 'rhomb_T3', 'rhomb_T4', 'rhomb_T5', 'rhomb_C7_L', 'rhomb_T1_L', 'rhomb_T2_L', 'rhomb_T3_L', 'rhomb_T4_L', 'rhomb_T5_L', 'sternohyoid', 'sternohyoid_L', 'omohyoid_inf', 'omohyoid_inf_L', 'omohyoid_sup', 'omohyoid_sup_L', 'rectcap_post_maj', 'rectcap_post_maj_L', 'rectcap_post_min', 'rectcap_post_min_L', 'obl_cap_sup', 'obl_cap_sup_L', 'obl_cap_inf', 'obl_cap_inf_L', 'long_col_c1thx', 'long_col_c1thx_L', 'longissi_cerv_c4thx', 'longissi_cerv_c4thx_L', 'longissi_cap_c6', 'longissi_cap_c6_L', 'long_cap_c4', 'long_cap_c4_L', 'rect_cap_lat', 'rect_cap_lat_L', 'calcn_r_right_Fx', 'calcn_r_right_Fy', 'calcn_r_right_Fz', 'calcn_r_right_px', 'calcn_r_right_py', 'calcn_r_right_pz', 'calcn_r_right_Tx', 'calcn_r_right_Ty', 'calcn_r_right_Tz', 'calcn_l_left_Fx', 'calcn_l_left_Fy', 'calcn_l_left_Fz', 'calcn_l_left_px', 'calcn_l_left_py', 'calcn_l_left_pz', 'calcn_l_left_Tx', 'calcn_l_left_Ty', 'calcn_l_left_Tz']

In [9]:
nums = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14]
#Список тех движений, для которых производится анализ
nums = [1, 2, 3, 7, 8, 9, 12]
time = {1: [-1, 2.3], 2: [-1, 2.5], 3: [-1, 1.3], 7: [-1, 4.4], 8: [-1, 6.3], 9: [-1, 1.8], 12: [-1, 100]}
for num in nums:
    #Префикс, который добавляется к названиям файлов
    prefix = 'SO_new\\final\\' + str(num) + '\\'
    #Название файла со скоростями
    velfile = prefix + '4_1_75_scaled_opt_N2.osim_muscles_changed_Actuation_speed'
    #Название файла с силами
    forcefile = prefix + '4_1_75_scaled_opt_N2.osim_muscles_changed_StaticOptimization_force'
    #Название файла с активациями
    actfile = prefix + '4_1_75_scaled_opt_N2.osim_muscles_changed_StaticOptimization_activation'
    
    #Содержание файлов в текстовой строке
    veltextdatalines = ''
    forcetextdatalines = ''
    acttextdatalines = ''
    
    #Чтение файлов
    with open (velfile, 'r') as fp:
        veldatalines = fp.readlines()
    with open (forcefile, 'r') as fp:
        forcedatalines = fp.readlines()
    with open (actfile, 'r') as fp:
        actdatalines = fp.readlines()
    
    #при чтении опускается ненужная техзничсекая информация
    for i in veldatalines[23:-1]:
        veltextdatalines += i + '\n'
    for i in forcedatalines[15:-1]:
        forcetextdatalines += i + '\n'
    for i in actdatalines[9:-1]:
        acttextdatalines += i + '\n'   
    
    #Перевод в формат dataframe
    vdf = pd.read_csv(StringIO(veltextdatalines), sep = '\t', names = muscles_listav, header = None)
    fdf = pd.read_csv(StringIO(forcetextdatalines), sep = '\t', names = muscles_listf, header = None)
    adf = pd.read_csv(StringIO(acttextdatalines), sep = '\t', names = muscles_listav, header = None)

    #Обрезание по времени
    start = time[num][0]
    end = time[num][1]
    vdf = vdf[vdf['time'] <= end]
    vdf = vdf[vdf['time'] >= start]
    fdf = fdf[fdf['time'] <= end]
    fdf = fdf[fdf['time'] >= start]
    adf = adf[adf['time'] <= end]
    adf = adf[adf['time'] >= start]
    
    #Запись результатлов в файлы
    newtextlines = ''.join(veldatalines[0:23]) + vdf.to_csv(sep = '\t', header=False, index = False)
    with open ('SO_new\\reduced data\\' + str(num) + '_Velocity_reduced.txt', 'w+') as fp:
        fp.write(newtextlines)
    newtextlines = ''.join(forcedatalines[0:15]) + fdf.to_csv(sep = '\t', header=False, index = False)
    with open ('SO_new\\reduced data\\' + str(num) + '_Force_reduced.txt', 'w+') as fp:
        fp.write(newtextlines)
    newtextlines = ''.join(veldatalines[0:9]) + adf.to_csv(sep = '\t', header=False, index = False)
    with open ('SO_new\\reduced data\\' + str(num) + '_Actuation_reduced.txt', 'w+') as fp:
        fp.write(newtextlines)
        